In [1]:
from imagenet_val import ValData
from interp_utils import *
from convnext import get_convnext
import torch

model = get_convnext()
model.eval().cuda()
val = ValData()

/home/noa/.miniconda/envs/convnext/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
from interp_utils import register_hook, caching_hook, remove_hooks
from tqdm import tqdm

remove_hooks(model, quiet=True)
for block in model.blocks:
    register_hook(block.act, caching_hook)



maxes, mins, means = [], [], []
for batch in tqdm(val.as_batches(20), total=len(val)//20):
    with torch.no_grad():
        data = batch.data.cuda()
        model(data)
        del data
    max_acts = [model.blocks[i].act.cache['input'].max(dim=1).values.max(dim=1).values.T for i in range(len(model.blocks))]
    max_acts = torch.stack(max_acts).cpu()
    maxes.append(max_acts)

    min_acts = [model.blocks[i].act.cache['input'].min(dim=1).values.min(dim=1).values.T for i in range(len(model.blocks))]
    min_acts = torch.stack(min_acts).cpu()
    mins.append(min_acts)

    mean_acts = [model.blocks[i].act.cache['input'].mean(dim=(1,2)).T for i in range(len(model.blocks))]
    mean_acts = torch.stack(mean_acts).cpu()
    means.append(mean_acts)



100%|██████████| 2500/2500 [04:17<00:00,  9.72it/s]


In [3]:
neuron_mins = torch.cat(mins, dim=-1)
torch.save(neuron_mins, 'val_set_neuron_mins.pt')

neuron_maxes = torch.cat(maxes, dim=-1)
torch.save(neuron_maxes, 'val_set_neuron_maxes.pt')

neuron_means = torch.cat(means, dim=-1)
torch.save(neuron_means, 'val_set_neuron_means.pt')


max_positivity = (neuron_maxes > 0.0).float().mean(dim=-1)
torch.save(max_positivity, 'max_positivity.pt')

min_positivity = (neuron_mins > 0.0).float().mean(dim=-1)
torch.save(min_positivity, 'min_positivity.pt')

mean_positivity = (neuron_means > 0.0).float().mean(dim=-1)
torch.save(mean_positivity, 'mean_positivity.pt')


In [ ]:
from interp_utils import TensorHistogramObserver

observer = TensorHistogramObserver(min=-10, max=4, bin_width=0.2, tensor_shape=(18, 1536,))
neuron_counts = torch.load('../gelu_investigation/neuron_bin_counts.pt')
observer.counts = neuron_counts

zero_idx = (observer.boundaries < 0.001).nonzero().max()
max_count = observer.counts[0].sum(dim=-1).int()
pw_positivity = observer.counts[:,:,zero_idx:].sum(dim=-1)/max_count

torch.save(pw_positivity, 'pw_positivity.pt')